## Preprocessing

In [1]:
import numpy as np

### Identify Audio Files

Obtain audio file names and tempos from a csv.

Audio files and the csv file are expected to be in the directory `data/`,  
and the csv file is expected to have the names of the audio file in the first column,  
and the tempos corresponding audio files in the second column,  
with no header.

In [3]:
dataf_dir = "data"
dataf_name = "data.csv"

In [4]:
# prepend directory to dataf_name if not present
dataf_name = dataf_dir + '/' + dataf_name if dataf_name.split('/')[0] != dataf_dir else dataf_name

audiofs = np.genfromtxt(dataf_name, delimiter=',', dtype='unicode')
# prepend directory to track names if not present
audiof_names = [dataf_dir + '/' + audiof_name if audiof_name.split('/')[0] != dataf_dir
                else audiof_name
                for audiof_name in audiofs[:, 0]]
# append file extension (.wav) to track names if not present
audiof_names = [audiof_name + ".wav" if audiof_name.split('.')[-1] != "wav"
                else audiof_name
                for audiof_name in audiof_names]
audiofs = np.stack((audiof_names, audiofs[:,1]), axis=1)

### Import Audio

In [50]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.interpolate import interp1d
from scipy.signal import stft, istft

In [97]:
mid_tempo = int(np.median(audiofs[:,1].astype(int)))
clips = []
for [audiof_name, tempo] in audiofs:
    rate, audiof = wavfile.read(audiof_name)
    # convert stereo to mono
    if audiof.shape[1] == 2:
        # type conversions to minimize risk of overflow
        audiof = np.int16(audiof.astype(int).sum(axis=1) // 2)
    # normalize tempo to the median of the dataset
    tempo = int(tempo)
    if tempo != mid_tempo:
        audio_length = audiof.shape[0] / rate
        intervals_old = np.linspace(0, audio_length, audiof.shape[0])
        intervals_new = np.linspace(0, audio_length, int(audiof.shape[0]*tempo/mid_tempo))
        # construct interpolation and resample
        audiof = np.int16(interp1d(intervals_old, audiof)(intervals_new))
    #TODO: produce random 10-second clips from audiof
        pass
        #clips.append(clip)
    print("Imported " + audiof_name)

Imported data/Adventure/Isometric.wav
Imported data/Adventure/You're On.wav
Imported data/Adventure/OK.wav
Imported data/Adventure/La Lune.wav
Imported data/Adventure/Pay No Mind.wav
Imported data/Adventure/Beings.wav
Imported data/Adventure/Imperium.wav
Imported data/Adventure/Zephyr.wav
Imported data/Adventure/Nonsense.wav
Imported data/Adventure/Innocence.wav
Imported data/Adventure/Pixel Empire.wav
Imported data/Adventure/Home.wav
Imported data/Adventure/Icarus.wav
Imported data/Adventure/Finale.wav
Imported data/Adventure/The City.wav
Imported data/Adventure/Cut the Kid.wav
Imported data/Adventure/Technicolor.wav
Imported data/Adventure/Only Way Out.wav
Imported data/Clarity/Hourglass.wav
Imported data/Clarity/Shave It Up.wav
Imported data/Clarity/Spectrum.wav
Imported data/Clarity/Lost At Sea.wav
Imported data/Clarity/Clarity.wav
Imported data/Clarity/Codec.wav
Imported data/Clarity/Stache.wav
Imported data/Clarity/Fall Into the Sky.wav
Imported data/Clarity/Follow You Down.wav
I

In [19]:
##test

rate, lalune = wavfile.read("data/Adventure/La Lune.wav")

lalune = np.int16(lalune.astype(int).sum(axis=1) // 2)

tempo = 70
mid_tempo = 125
audio_length = audiof.shape[0] / rate
intervals_old = np.linspace(0, audio_length, lalune.shape[0])
intervals_new = np.linspace(0, audio_length, int(lalune.shape[0]*tempo/mid_tempo))
lalune = np.int16(interp1d(intervals_old, lalune)(intervals_new))

In [86]:
##test
print(np.fft.fft(lalune).shape)
print(lalune.shape)

(5429840,)
(5429840,)


## Build Network

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
model = Sequential()